In [1]:
import pandas as pd
import sklearn
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

import numpy as np

In [2]:
crime_property_light_population = pd.read_csv("./model_data/crime_light_density.csv")
crime_property_light_population.fillna(method='ffill', inplace=True)
# crime_property_light_population.dropna(inplace=True)
crime_property_light_population.head()

,Unnamed: 0,OFFENSE_CODE_GROUP,DISTRICT,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,zipcode,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density
0,0,Auto Theft,E18,N,2019-10-13 09:28:24,2019,10,Sunday,9,LINCOLN ST,42.259518,-71.121563,2136,6206.9,455.644447,21.073507,0.437849
1,1,Auto Theft,E18,N,2019-10-12 20:11:26,2019,10,Saturday,20,METROPOLITAN AVE,42.262092,-71.116710,2136,6206.9,455.644447,21.073507,0.437849
2,2,Auto Theft,D14,N,2019-10-12 15:12:43,2019,10,Saturday,15,ALLSTON ST,42.352375,-71.135096,2134,16212.0,2250.588503,63.554783,0.582893
3,3,Auto Theft,D4,N,2019-10-12 04:41:52,2019,10,Saturday,4,SAINT JAMES AVE,42.349476,-71.076402,2116,32724.1,14799.401260,480.463801,0.656585
4,4,Auto Theft,A15,N,2019-10-02 08:08:49,2019,10,Wednesday,8,N MEAD ST,42.381846,-71.066551,2129,12192.0,278.377510,165.249738,0.678941


In [3]:
predictors = ['OFFENSE_CODE_GROUP','SHOOTING', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'Population density (per square mile of land area)', 'BLDG_VAL', 'LAND_VAL','light_density', 'Lat', 'Long']
# predictors = ['OFFENSE_CODE_GROUP','SHOOTING', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'Population density (per square mile of land area)', 'BLDG_VAL', 'LAND_VAL','light_density']

df = crime_property_light_population[predictors]



print(len(pd.DataFrame(df.OFFENSE_CODE_GROUP.value_counts())))
print(pd.DataFrame(df.OFFENSE_CODE_GROUP.value_counts()).head(10))

# crime_type = ['Auto Theft', 'Motor Vehicle Accident Response', 'Larceny', 'Robbery', 'Residential Burglary']

# df = df[df['OFFENSE_CODE_GROUP'].isin(crime_type)]

# count of each type before balancing
# print(df.OFFENSE_CODE_GROUP.value_counts())

property_crime = ["Larceny", "Larceny From Motor Vehicle"]
violent_crime = ["Simple Assault", "Aggravated Assault", "Robbery", "Homicide", "Sexual Assault"]


df_property_crime = df[df['OFFENSE_CODE_GROUP'].isin(property_crime)]
df_property_crime= df_property_crime.assign(OFFENSE_CODE_GROUP='Property Crime')

66
                                 OFFENSE_CODE_GROUP
Motor Vehicle Accident Response               40471
Larceny                                       33610
Medical Assistance                            31105
Investigate Person                            23700
Other                                         22575
Simple Assault                                20002
Vandalism                                     19376
Drug Violation                                19205
Verbal Disputes                               17538
Investigate Property                          14395


In [4]:
df_motor_vehicle_accident = df[df['OFFENSE_CODE_GROUP'] == 'Motor Vehicle Accident Response']

df_violent = df[df['OFFENSE_CODE_GROUP'].isin(violent_crime)]
df_violent= df_violent.assign(OFFENSE_CODE_GROUP='Violent Crime')

df_medical = df[df['OFFENSE_CODE_GROUP'] == 'Medical Assistance']
df_medical= df_medical.assign(OFFENSE_CODE_GROUP='Medical Assistance')

df_drug = df[df['OFFENSE_CODE_GROUP'] == 'Drug Violation']
# df_vandal = df[df['OFFENSE_CODE_GROUP'] == 'Vandalism']

df = pd.concat([df_property_crime, df_motor_vehicle_accident, df_violent, df_medical, df_drug])
# df = pd.concat([df_property_crime, df_motor_vehicle_accident, df_assault, df_medical, df_vandal])

In [5]:
print(df.shape)
df.OFFENSE_CODE_GROUP.value_counts()

(172921, 11)


Property Crime                     46743
Motor Vehicle Accident Response    40471
Violent Crime                      35397
Medical Assistance                 31105
Drug Violation                     19205
Name: OFFENSE_CODE_GROUP, dtype: int64

In [6]:
# add light data's lat long
# add graffiti data's lat long
# analyze the max and min of the concatenated lat long respectively
# bin it
# merge with binning 
from sklearn.preprocessing import KBinsDiscretizer


#Western most coord: 42.281549, -71.191510
#Eastern most: 42.325468, -70.922188
#southern most: 42.227155, -71.130342
#northern most: 42.397988, -71.000505
grid_lat = np.linspace(42.2267, 42.4, 192, endpoint='False')
grid_lat_pd = pd.DataFrame(grid_lat, columns=["Lat"])

grid_long = np.linspace(-71.192, -70.9225, 220, endpoint='False')
grid_long_pd = pd.DataFrame(grid_long, columns=["Long"])
#Boston City
#21.96 km wide ==> 220 bins , 100m grid
#19.20 km long ==> 192 bins, 100m grid
#########################
lat_bin_count = 192
long_bin_count = 220
#########################
lat_enc = KBinsDiscretizer(n_bins=lat_bin_count,
                           strategy='uniform', encode='ordinal')
long_enc = KBinsDiscretizer(n_bins=long_bin_count,
                            strategy='uniform', encode='ordinal')






# Boston City's Rectangular boundary:
# lat : 42.227155 ~ 42.397988
# long : -71.191510 ~ -70.922188
boston_west = -71.191512
boston_east = -70.922185
boston_north = 42.397990
boston_south = 42.227150
df = df[df['Lat'] <= 42.397990]
df = df[df['Lat'] >= 42.227150]
df = df[df['Long'] <= -70.922185]
df = df[df['Long'] >= -71.191512]
print(len(df)) # result : 147858 ==> we don't lose any data (all rows are accurate)
# df["latbin"] = lat_enc.fit_transform(pd.DataFrame(df.Lat))
lat_enc.fit(pd.DataFrame(grid_lat_pd.Lat))
long_enc.fit(pd.DataFrame(grid_long_pd.Long))

df["latbin"] = lat_enc.transform(pd.DataFrame(df.Lat))
df["longbin"] = long_enc.transform(pd.DataFrame(df.Long))

# df["lonbin"] = df.Longitude.map(to_bin)
# groups = df.groupby(("latbin", "lonbin"))
# print((boston_north - boston_south) / 192.0  * 161 + boston_south)
df.head()

172921


,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,Lat,Long,latbin,longbin
21,Property Crime,N,9,Sunday,3,28541.5,62047.075801,297.436153,0.816495,42.340070,-71.052794,126.0,114.0
47,Property Crime,N,9,Saturday,23,19926.9,560.432683,189.333050,1.000000,42.344396,-71.086320,130.0,86.0
87,Property Crime,N,9,Saturday,20,10617.5,1409.245482,46.711201,0.529223,42.319293,-71.111782,103.0,66.0
90,Property Crime,N,9,Wednesday,4,14884.4,1040.984096,31.260815,0.640733,42.306587,-71.067479,88.0,102.0
93,Property Crime,N,9,Saturday,19,15913.2,1293.230035,31.015855,0.568578,42.291265,-71.068114,72.0,101.0


In [7]:
light = pd.read_csv('./model_data/streetlight_locations.csv')
light = light[["Lat", "Long"]]

#########
lat_long_multiplier = 2 # 2 : 40.6

lat_bin_count3 = lat_bin_count * lat_long_multiplier
long_bin_count3 = long_bin_count * lat_long_multiplier
lat_enc3 = KBinsDiscretizer(n_bins=lat_bin_count3, strategy='uniform', encode='ordinal')
long_enc3 = KBinsDiscretizer(n_bins=long_bin_count3, strategy='uniform', encode='ordinal')

lat_enc3.fit(pd.DataFrame(grid_lat_pd.Lat))
long_enc3.fit(pd.DataFrame(grid_long_pd.Long))

df["latbin3"] = lat_enc3.transform(pd.DataFrame(df.Lat))
df["longbin3"] = long_enc3.transform(pd.DataFrame(df.Long))
########

light["latbin3"] =  lat_enc3.transform(pd.DataFrame(light.Lat))
light["longbin3"] =  long_enc3.transform(pd.DataFrame(light.Long))
light.head()




light_count = pd.DataFrame(light.groupby(['latbin3', 'longbin3']).size())


light_count.columns = ['light_count']

light_count.to_csv("light_count.csv")
light2 = pd.read_csv('./light_count.csv')

print(light2["light_count"].max())

df = df.join(light2.set_index(['latbin3', 'longbin3']), on=['latbin3', 'longbin3'])
df["light_count"] = df["light_count"].fillna(0)
df.head()

24


,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,Lat,Long,latbin,longbin,latbin3,longbin3,light_count
21,Property Crime,N,9,Sunday,3,28541.5,62047.075801,297.436153,0.816495,42.340070,-71.052794,126.0,114.0,252.0,228.0,5.0
47,Property Crime,N,9,Saturday,23,19926.9,560.432683,189.333050,1.000000,42.344396,-71.086320,130.0,86.0,261.0,173.0,2.0
87,Property Crime,N,9,Saturday,20,10617.5,1409.245482,46.711201,0.529223,42.319293,-71.111782,103.0,66.0,206.0,132.0,6.0
90,Property Crime,N,9,Wednesday,4,14884.4,1040.984096,31.260815,0.640733,42.306587,-71.067479,88.0,102.0,177.0,204.0,7.0
93,Property Crime,N,9,Saturday,19,15913.2,1293.230035,31.015855,0.568578,42.291265,-71.068114,72.0,101.0,144.0,203.0,1.0


# Below is grid scaling for bldg value ==> not so effective

In [8]:
# bldg = pd.read_csv('./model_data/union_2019_building_value.csv')
# bldg = bldg[["BLDG_VAL", "0", "1"]]
# bldg = bldg.dropna()
# bldg = bldg.rename(columns={"BLDG_VAL": "avg_BLDG_val", "0": "latitude", "1":"longitude"})
# print(len(bldg))
# bldg["latitude"] = bldg["latitude"].astype(float)
# # bldg["latitude"] = pd.to_numeric(df["latitude"],errors='coerce')
# bldg["longitude"] = bldg["longitude"].astype(float)


# bldg["latbin"] =  lat_enc.transform(pd.DataFrame(bldg.latitude))
# bldg["longbin"] =  long_enc.transform(pd.DataFrame(bldg.longitude))

# print(bldg['latbin'].unique())

# print(bldg[bldg['latbin']<42] .head())

# bldg_val_avg = pd.DataFrame(bldg.groupby(['latbin', 'longbin']).mean())
# bldg_val_avg.drop(['latitude', 'longitude' ], axis=1, inplace=True)
# print(bldg_val_avg.head())

# bldg_val_avg.to_csv("2019_avg_bldg_val.csv")

# bldg_val_avg2 = pd.read_csv('./2019_avg_bldg_val.csv')

# df = df.join(bldg_val_avg2.set_index(['latbin', 'longbin']), on=['latbin', 'longbin'])

# df["avg_BLDG_val"] = df["avg_BLDG_val"].fillna(0)
# df.head()

In [9]:
graffiti = pd.read_csv('./clean_data/boston_graffiti.csv')
graffiti = graffiti[graffiti["reason"] == 'Graffiti' ]
# graffiti.head()

#########
bin_multiplier = 3

lat_bin_count4 = lat_bin_count // bin_multiplier
long_bin_count4 = long_bin_count // bin_multiplier
lat_enc4 = KBinsDiscretizer(n_bins=lat_bin_count4, strategy='uniform', encode='ordinal')
long_enc4 = KBinsDiscretizer(n_bins=long_bin_count4, strategy='uniform', encode='ordinal')
lat_enc4.fit(pd.DataFrame(grid_lat_pd.Lat))
long_enc4.fit(pd.DataFrame(grid_long_pd.Long))
df["latbin4"] = lat_enc4.transform(pd.DataFrame(df.Lat))
df["longbin4"] = long_enc4.transform(pd.DataFrame(df.Long))
########


graffiti["latbin4"] =  lat_enc4.transform(pd.DataFrame(graffiti.latitude))
graffiti["longbin4"] =  long_enc4.transform(pd.DataFrame(graffiti.longitude))
graffiti.head()

print(graffiti['latbin4'].unique())

graffiti_count = pd.DataFrame(graffiti.groupby(['latbin4', 'longbin4']).size())

graffiti_count.columns = ['graffiti_count']

print(graffiti_count.head())
graffiti_count.to_csv("graffiti_count.csv")

[30. 27. 43. 29. 49. 45. 28. 54. 25. 24. 35. 46. 44. 26. 50. 51. 42. 57.
 36. 47. 48. 31. 34. 55. 33. 41. 40. 58. 60. 32. 52. 21. 22. 59. 38. 16.
 23. 39.  7. 19. 10. 15. 17. 13. 37. 14.  6.  4. 56. 20.  5. 18. 11. 53.
  8.  9. 12. 62.  3. 61.  1.]
                  graffiti_count
latbin4 longbin4                
1.0     15.0                   1
3.0     16.0                   4
        17.0                   2
4.0     15.0                   4
        16.0                   9


In [10]:
graffiti2 = pd.read_csv('./graffiti_count.csv')

print(graffiti2["graffiti_count"].max())

df = df.join(graffiti2.set_index(['latbin4', 'longbin4']), on=['latbin4', 'longbin4'])

df["graffiti_count"] = df["graffiti_count"].fillna(0)
df.head()

4471


,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,Lat,Long,latbin,longbin,latbin3,longbin3,light_count,latbin4,longbin4,graffiti_count
21,Property Crime,N,9,Sunday,3,28541.5,62047.075801,297.436153,0.816495,42.340070,-71.052794,126.0,114.0,252.0,228.0,5.0,42.0,37.0,68.0
47,Property Crime,N,9,Saturday,23,19926.9,560.432683,189.333050,1.000000,42.344396,-71.086320,130.0,86.0,261.0,173.0,2.0,43.0,28.0,363.0
87,Property Crime,N,9,Saturday,20,10617.5,1409.245482,46.711201,0.529223,42.319293,-71.111782,103.0,66.0,206.0,132.0,6.0,34.0,21.0,52.0
90,Property Crime,N,9,Wednesday,4,14884.4,1040.984096,31.260815,0.640733,42.306587,-71.067479,88.0,102.0,177.0,204.0,7.0,29.0,33.0,13.0
93,Property Crime,N,9,Saturday,19,15913.2,1293.230035,31.015855,0.568578,42.291265,-71.068114,72.0,101.0,144.0,203.0,1.0,24.0,33.0,2.0


In [11]:
boston_shape = pd.read_csv("./clean_data/boston_shape_interpolation_data.csv")
boston_shape.head()
boston_geo_lat_long = boston_shape[["GEOID10","INTPTLAT10","INTPTLON10"]]

boston_geo_lat_long.head()

boston_social_vulnerability = pd.read_csv("./clean_data/Climate_Ready_Boston_Social_Vulnerability.csv")

# boston_social_features =["GEOID10","Low_to_No"]
boston_social_features =["GEOID10","HU100_RE", "Low_to_No"]

boston_low_income = boston_social_vulnerability[boston_social_features]

boston_low_income.head()

boston_low_income = boston_low_income.join(boston_geo_lat_long.set_index(['GEOID10']), on=['GEOID10'])
# boston_low_income.drop(['POP100_RE'], axis=1, inplace=True)

boston_low_income.rename(columns = {'INTPTLAT10':'lat', 'INTPTLON10':'long'}, inplace = True) 

boston_low_income.head()
########################
lat_long_divider = 10 # 20 3623

lat_bin_count2 = lat_bin_count // lat_long_divider
long_bin_count2 = long_bin_count // lat_long_divider

lat_enc2 = KBinsDiscretizer(n_bins=lat_bin_count2, strategy='uniform', encode='ordinal')
long_enc2 = KBinsDiscretizer(n_bins=long_bin_count2, strategy='uniform', encode='ordinal')
lat_enc2.fit(pd.DataFrame(grid_lat_pd.Lat))
long_enc2.fit(pd.DataFrame(grid_long_pd.Long))
df["latbin2"] = lat_enc2.transform(pd.DataFrame(df.Lat))
df["longbin2"] = long_enc2.transform(pd.DataFrame(df.Long))
#########################



boston_low_income["latbin2"] =  lat_enc2.transform(pd.DataFrame(boston_low_income.lat))
boston_low_income["longbin2"] =  long_enc2.transform(pd.DataFrame(boston_low_income.long))


# print(boston_low_income['latbin2'].unique())
# print(boston_low_income['longbin2'].unique())




low_income_pop = pd.DataFrame(boston_low_income.groupby(['latbin2', 'longbin2']).mean())
low_income_pop.drop(['lat', 'long' ], axis=1, inplace=True)
low_income_pop.drop(['GEOID10'], axis=1, inplace=True)
print(low_income_pop)

low_income_pop.to_csv("low_income_pop_dist.csv")

low_income_pop2 = pd.read_csv('./low_income_pop_dist.csv')

# low_income_pop2.loc[low_income_pop2['latbin'] == 8]

                  HU100_RE  Low_to_No
latbin2 longbin2                     
1.0     5.0          935.0      160.0
2.0     4.0         1654.0      666.0
        6.0         2081.0      840.0
3.0     3.0         2192.0     1911.0
        5.0         1051.0      322.0
...                    ...        ...
17.0    13.0        1446.0     1587.0
        14.0        1784.0      939.0
        15.0        2345.0     2013.0
        16.0           0.0        0.0
18.0    15.0           3.0        0.0

[105 rows x 2 columns]


In [12]:
df = df.join(low_income_pop2.set_index(['latbin2', 'longbin2']), on=['latbin2', 'longbin2'])
df.head()

# print(df.loc[df['Low_to_No'].isna() == False].shape)
# df = df.loc[df['Low_to_No'].isna() == False]

,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,Lat,...,latbin3,longbin3,light_count,latbin4,longbin4,graffiti_count,latbin2,longbin2,HU100_RE,Low_to_No
21,Property Crime,N,9,Sunday,3,28541.5,62047.075801,297.436153,0.816495,42.340070,...,252.0,228.0,5.0,42.0,37.0,68.0,12.0,11.0,1338.500000,879.000000
47,Property Crime,N,9,Saturday,23,19926.9,560.432683,189.333050,1.000000,42.344396,...,261.0,173.0,2.0,43.0,28.0,363.0,12.0,8.0,1541.000000,1552.666667
87,Property Crime,N,9,Saturday,20,10617.5,1409.245482,46.711201,0.529223,42.319293,...,206.0,132.0,6.0,34.0,21.0,52.0,10.0,6.0,732.000000,223.000000
90,Property Crime,N,9,Wednesday,4,14884.4,1040.984096,31.260815,0.640733,42.306587,...,177.0,204.0,7.0,29.0,33.0,13.0,8.0,10.0,1302.666667,1752.333333
93,Property Crime,N,9,Saturday,19,15913.2,1293.230035,31.015855,0.568578,42.291265,...,144.0,203.0,1.0,24.0,33.0,2.0,7.0,10.0,1227.000000,692.000000


In [13]:
# df["POP100_RE"] = df["POP100_RE"].fillna(0)
df["HU100_RE"] = df["HU100_RE"].fillna(0)
df["Low_to_No"] = df["Low_to_No"].fillna(0)
# df["LEP"] = df["LEP"].fillna(0)
# df["POC2"] = df["POC2"].fillna(0)


# display(df.head(10))

# df['Low_to_No'] = df.groupby(['latbin','longbin']).Low_to_No.transform(lambda x: x.interpolate())
# df["POP100_RE"] = df.groupby(['latbin','longbin']).Low_to_No.transform(lambda x: x.interpolate())
# df["HU100_RE"] = df.groupby(['latbin','longbin']).Low_to_No.transform(lambda x: x.interpolate())

# df["POP100_RE"] = df["POP100_RE"].fillna(0)
# df["HU100_RE"] = df["HU100_RE"].fillna(0)
# df["Low_to_No"] = df["Low_to_No"].fillna(0)


df.head(10)


#POP100_RE: Tract population count
#HU100_RE: Tract housing unit count
#Low_to_No: # of population that has low to no income

,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,Lat,...,latbin3,longbin3,light_count,latbin4,longbin4,graffiti_count,latbin2,longbin2,HU100_RE,Low_to_No
21,Property Crime,N,9,Sunday,3,28541.5,62047.075801,297.436153,0.816495,42.340070,...,252.0,228.0,5.0,42.0,37.0,68.0,12.0,11.0,1338.500000,879.000000
47,Property Crime,N,9,Saturday,23,19926.9,560.432683,189.333050,1.000000,42.344396,...,261.0,173.0,2.0,43.0,28.0,363.0,12.0,8.0,1541.000000,1552.666667
87,Property Crime,N,9,Saturday,20,10617.5,1409.245482,46.711201,0.529223,42.319293,...,206.0,132.0,6.0,34.0,21.0,52.0,10.0,6.0,732.000000,223.000000
90,Property Crime,N,9,Wednesday,4,14884.4,1040.984096,31.260815,0.640733,42.306587,...,177.0,204.0,7.0,29.0,33.0,13.0,8.0,10.0,1302.666667,1752.333333
93,Property Crime,N,9,Saturday,19,15913.2,1293.230035,31.015855,0.568578,42.291265,...,144.0,203.0,1.0,24.0,33.0,2.0,7.0,10.0,1227.000000,692.000000
95,Property Crime,N,9,Saturday,19,15913.2,1293.230035,31.015855,0.568578,42.295263,...,152.0,196.0,5.0,25.0,32.0,21.0,7.0,9.0,1077.000000,793.000000
96,Property Crime,N,9,Saturday,19,32724.1,14799.401260,480.463801,0.656585,42.347972,...,269.0,186.0,5.0,44.0,30.0,45.0,13.0,9.0,2010.000000,503.000000
108,Property Crime,N,9,Saturday,18,11504.8,271.853366,28.739445,0.529698,42.267288,...,90.0,125.0,0.0,15.0,20.0,0.0,4.0,6.0,2906.000000,1654.000000
118,Property Crime,N,9,Saturday,16,16212.0,2250.588503,63.554783,0.582893,42.349897,...,273.0,100.0,2.0,45.0,16.0,239.0,13.0,5.0,1418.000000,1740.000000
122,Property Crime,N,9,Saturday,17,15913.2,1293.230035,31.015855,0.568578,42.290575,...,142.0,197.0,7.0,23.0,32.0,31.0,7.0,9.0,1077.000000,793.000000


In [14]:
# data balancing
crime_MVAR = df[df['OFFENSE_CODE_GROUP']=="Motor Vehicle Accident Response"]
crime_property  = df[df['OFFENSE_CODE_GROUP']=="Property Crime"]
crime_other_crime = df[(df['OFFENSE_CODE_GROUP']!="Motor Vehicle Accident Response") & (df['OFFENSE_CODE_GROUP']!="Property Crime") & (df['OFFENSE_CODE_GROUP']!="Violent Crime") & (df['OFFENSE_CODE_GROUP']!="Medical Assistance")]
crime_medical = df[df['OFFENSE_CODE_GROUP']=="Medical Assistance"]
crime_violent = df[df['OFFENSE_CODE_GROUP']=="Violent Crime"]
t_size= 30000
#  keep about 6000 row for each crime
drop_MVAR, keep_MVAR = train_test_split(crime_MVAR, test_size=t_size/len(crime_MVAR), random_state = 109, stratify = crime_MVAR[['MONTH', 'DAY_OF_WEEK']])
drop_property, keep_property = train_test_split(crime_property, test_size=t_size/len(crime_property), random_state = 109, stratify = crime_property[['MONTH', 'DAY_OF_WEEK']])
drop_violent, keep_violent = train_test_split(crime_violent, test_size=t_size/len(crime_violent), random_state = 109, stratify = crime_violent[['MONTH', 'DAY_OF_WEEK']])
drop_medical, keep_medical = train_test_split(crime_medical, test_size=t_size/len(crime_medical), random_state = 109, stratify = crime_medical[['MONTH', 'DAY_OF_WEEK']])

# combine data
balanced_data = pd.concat([keep_MVAR, keep_property, keep_violent, keep_medical, crime_other_crime])

# count of each type after balancing 
balanced_data.OFFENSE_CODE_GROUP.value_counts()

Property Crime                     30000
Motor Vehicle Accident Response    30000
Medical Assistance                 30000
Violent Crime                      30000
Drug Violation                     19205
Name: OFFENSE_CODE_GROUP, dtype: int64

In [15]:
# balanced_data

In [16]:
#Let's label encode & one-hot encode the Categoricals (SHOOTING, DAY_OF_WEEK)
# df = balanced_data

In [17]:


enc = OneHotEncoder(handle_unknown='ignore')

le = LabelEncoder() 
  
df['SHOOTING']= le.fit_transform(df['SHOOTING'])
df['DAY_OF_WEEK']= le.fit_transform(df['DAY_OF_WEEK']) 
df['OFFENSE_CODE_GROUP'] = le.fit_transform(df['OFFENSE_CODE_GROUP'])
df.reset_index(drop=True, inplace=True)

df_cat = enc.fit_transform(df[["SHOOTING", "MONTH", "DAY_OF_WEEK", "HOUR"]]).toarray()
df_cat = pd.DataFrame(df_cat, columns=enc.get_feature_names(['SHOOTING', 'MONTH', 'DAY_OF_WEEK', 'HOUR']))
df_cat.reset_index(drop=True, inplace=True)

df = pd.concat([df, df_cat], axis=1)
# print(df_cat.head())
df.head()

,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,Lat,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,3,0,9,3,3,28541.5,62047.075801,297.436153,0.816495,42.340070,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0,9,2,23,19926.9,560.432683,189.333050,1.000000,42.344396,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,0,9,2,20,10617.5,1409.245482,46.711201,0.529223,42.319293,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3,0,9,6,4,14884.4,1040.984096,31.260815,0.640733,42.306587,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,0,9,2,19,15913.2,1293.230035,31.015855,0.568578,42.291265,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# Dropping duplicate features (due to grid scaling)

In [18]:
df.drop(['light_density'], axis=1, inplace=True)
df.drop(['latbin'], axis=1, inplace=True)
df.drop(['longbin'], axis=1, inplace=True)
df.drop(['latbin2'], axis=1, inplace=True)
df.drop(['longbin2'], axis=1, inplace=True)
df.drop(['latbin3'], axis=1, inplace=True)
df.drop(['longbin3'], axis=1, inplace=True)
df.drop(['latbin4'], axis=1, inplace=True)
df.drop(['longbin4'], axis=1, inplace=True)
# df.drop(['BLDG_VAL'], axis=1, inplace=True)
df.drop(['SHOOTING'], axis=1, inplace=True)


# df.drop(['SHOOTING_1', 'HOUR_1','HOUR_3', 'HOUR_4', 'HOUR_5', 'HOUR_6'], axis=1, inplace=True)





In [19]:
df.head()

,OFFENSE_CODE_GROUP,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,Lat,Long,light_count,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,3,9,3,3,28541.5,62047.075801,297.436153,42.340070,-71.052794,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,9,2,23,19926.9,560.432683,189.333050,42.344396,-71.086320,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,9,2,20,10617.5,1409.245482,46.711201,42.319293,-71.111782,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3,9,6,4,14884.4,1040.984096,31.260815,42.306587,-71.067479,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,9,2,19,15913.2,1293.230035,31.015855,42.291265,-71.068114,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [20]:
df.shape

(172921, 58)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'OFFENSE_CODE_GROUP'], 
                                                         df.OFFENSE_CODE_GROUP, test_size=0.2, 
                                                         random_state = 109, 
                                                         stratify = df.OFFENSE_CODE_GROUP)

In [22]:
#Let's scale the variables
min_max_scaler = MinMaxScaler()

# standard_scaler = StandardScaler()

# numerical = ["Population density (per square mile of land area)", "BLDG_VAL", "LAND_VAL", "light_density"]
to_standardize = ['Population density (per square mile of land area)','BLDG_VAL', 'LAND_VAL', 'Lat', 'Long', 'latbin', 'longbin', 'light_count',
       'avg_BLDG_val', 'graffiti_count']
not_standardize = list(df_cat.columns)
not_standardize.append('SHOOTING')
not_standardize.append('MONTH')
not_standardize.append('DAY_OF_WEEK')
not_standardize.append('HOUR')

min_max_scaler.fit(X_train)

# standard_scaler.fit(X_train[to_standardize])

X_normalized = pd.DataFrame(min_max_scaler.transform(X_train))
# X_normalized = pd.concat([pd.DataFrame(standard_scaler.transform(X_train[to_standardize])),
#                          pd.DataFrame(X_train[not_standardize])]
#                          , axis=1 )

X_normalized.columns = X_train.columns
X_normalized_tst = pd.DataFrame(min_max_scaler.transform(X_test))
# X_normalized_tst = pd.concat([pd.DataFrame(standard_scaler.transform(X_test[to_standardize])),
#                          pd.DataFrame(X_test[not_standardize])]
#                          , axis=1 )

X_normalized_tst.columns = X_test.columns


# X_normalized.drop(['SHOOTING'], axis=1, inplace=True)
# X_normalized_tst.drop(['SHOOTING'], axis=1, inplace=True)
X_normalized.drop(['MONTH'], axis=1, inplace=True)
X_normalized_tst.drop(['MONTH'], axis=1, inplace=True)
X_normalized.drop(['DAY_OF_WEEK'], axis=1, inplace=True)
X_normalized_tst.drop(['DAY_OF_WEEK'], axis=1, inplace=True)
X_normalized.drop(['HOUR'], axis=1, inplace=True)
X_normalized_tst.drop(['HOUR'], axis=1, inplace=True)
X_normalized.head()

,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,Lat,Long,light_count,graffiti_count,HU100_RE,Low_to_No,SHOOTING_0,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,0.082572,0.000127,0.009073,0.332138,0.113859,0.277778,0.000671,0.000000,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.090394,0.002940,0.000000,0.142707,0.247771,0.055556,0.004697,0.310671,0.125000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.236104,0.017019,0.056039,0.732549,0.236939,0.055556,0.027287,0.419155,0.675466,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.236459,0.008274,0.033911,0.677000,0.163497,0.222222,0.006263,0.000000,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.231752,0.009510,0.013115,0.355307,0.499670,0.222222,0.006934,0.318356,0.307842,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
X_normalized.drop(['SHOOTING_0', 'MONTH_1', 'DAY_OF_WEEK_0', 'HOUR_0'], axis=1, inplace=True)
X_normalized_tst.drop(['SHOOTING_0', 'MONTH_1', 'DAY_OF_WEEK_0', 'HOUR_0'], axis=1, inplace=True)
X_normalized

,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,Lat,Long,light_count,graffiti_count,HU100_RE,Low_to_No,SHOOTING_1,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,0.082572,0.000127,0.009073,0.332138,0.113859,0.277778,0.000671,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.090394,0.002940,0.000000,0.142707,0.247771,0.055556,0.004697,0.310671,0.125000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.236104,0.017019,0.056039,0.732549,0.236939,0.055556,0.027287,0.419155,0.675466,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.236459,0.008274,0.033911,0.677000,0.163497,0.222222,0.006263,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.231752,0.009510,0.013115,0.355307,0.499670,0.222222,0.006934,0.318356,0.307842,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138331,0.216769,0.007531,0.013439,0.492486,0.445742,0.333333,0.001342,0.472165,0.690088,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
138332,0.229294,0.080828,0.116790,0.526916,0.588374,0.055556,0.002908,0.308602,0.598602,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
138333,0.236104,0.017019,0.056039,0.759650,0.187342,0.222222,0.011854,0.504582,0.284161,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
138334,0.216769,0.007531,0.013439,0.475773,0.436953,0.111111,0.004697,0.472165,0.690088,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# Best so far Log : 32.33 ( bin factor :4)

In [24]:
# balanced data
# Let's first start with Logistic Model

logreg = LogisticRegression(random_state = 0, solver='lbfgs', multi_class='multinomial', max_iter = 1000)
logreg.fit(X_normalized, y_train)
y_pred = logreg.predict(X_normalized_tst)


mcm = multilabel_confusion_matrix(y_test, y_pred, sample_weight=None, labels=None, samplewise=False)
logreg.score(X_normalized_tst , y_test)

0.327627584212809

In [25]:
logreg.score(X_normalized, y_train)

0.32828764746703676

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 12).fit(X_normalized, y_train) 
  
# accuracy on X_test 
accuracy = knn.score(X_normalized_tst, y_test) 
print(accuracy)

In [ ]:
# decision tree with best depth by cv
depth_list = range(1,25)
score_list = []

for depth in depth_list:
#     print(depth)
    clf = DecisionTreeClassifier(random_state=0, criterion='gini', max_depth=depth)
    score = np.mean(cross_val_score(clf, X_normalized, y_train, cv=5))
    score_list.append(score)
plt.plot(depth_list, score_list)
plt.show()

In [ ]:
best_depth = depth_list[np.argmax(score_list)]
best_depth

In [ ]:
clf = DecisionTreeClassifier(random_state=0, max_depth=best_depth)
clf.fit(X_normalized, y_train)
clf.score(X_normalized_tst, y_test)

In [ ]:
# plot feature importances
plt.figure(figsize=(5,10))
features = X_normalized.columns
feature_importances = clf.feature_importances_
idx = np.argsort(feature_importances)
print(idx)
plt.barh(y=features[idx], width=feature_importances[idx])
plt.xlabel("Relative feature importance")
plt.show()